In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import string
import re

In [2]:
# Using Shakespeare text from TensorFlow datasets
import tensorflow_datasets as tfds

# Load the Shakespeare dataset
shakespeare_ds = tfds.load('tiny_shakespeare', split='train', as_supervised=False)
shakespeare_text = ""
for example in tfds.as_numpy(shakespeare_ds):
    shakespeare_text += example['text'].decode('utf-8')


In [3]:
# Print some statistics
print(f"Total characters: {len(shakespeare_text)}")
print(f"First 100 characters: {shakespeare_text[:100]}")

# Clean the text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    return text

text = clean_text(shakespeare_text)


Total characters: 1003854
First 100 characters: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# Create character mappings
chars = sorted(list(set(text)))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}
vocab_size = len(chars)

print(f'Unique characters: {vocab_size}')
print(f'Characters: {chars[:20]}...')


Unique characters: 38
Characters: [' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']...


In [5]:
# Create sequences
seq_length = 100
step = 1

sequences = []
next_chars = []

for i in range(0, len(text) - seq_length, step):
    sequences.append(text[i:i + seq_length])
    next_chars.append(text[i + seq_length])

print(f'Number of sequences: {len(sequences)}')


Number of sequences: 997452


In [6]:
# Vectorize sequences
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool_)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool_)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1


In [7]:
# Build the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(seq_length, vocab_size), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 128)       │        85,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │         4,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 221,990 (867.15 KB)

 Trainable params: 221,990 (867.15 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Function to sample the next character
def sample_next_char(preds, temperature=1.0):
    # Scale predictions by temperature
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # Sample based on probabilities
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Function to generate text
def generate_text(model, seed_text, num_chars=300, temperature=0.5):
    generated_text = seed_text

    for _ in range(num_chars):
        # Vectorize the current text
        x_pred = np.zeros((1, seq_length, vocab_size))
        for t, char in enumerate(seed_text):
            if char in char_to_idx:
                x_pred[0, t, char_to_idx[char]] = 1

        # Make a prediction
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample_next_char(preds, temperature)
        next_char = idx_to_char[next_index]

        # Append the predicted character
        generated_text += next_char
        seed_text = seed_text[1:] + next_char

    return generated_text


In [10]:
# Define checkpoint callback
checkpoint_path = "lstm_shakespeare_weights.weights.h5" # Changed the file name to include '.weights'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    monitor='loss',
    verbose=1
)


In [ ]:
# Train the model (with limited epochs for demonstration)
history = model.fit(
    X, y,
    batch_size=128,
    epochs=5,
    verbose=1,
    callbacks=[checkpoint_callback]
)

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()


In [ ]:
# Generate text with different temperatures
seed_texts = [
    "First Citizen:\nBefore we proceed any further, hear me speak.",
    "Hamlet:\nTo be, or not to be, that is the",
    "Romeo:\nBut, soft! what light through yonder"
]

temperatures = [0.2, 0.5, 1.0]

for seed_text in seed_texts:
    print(f"Seed text: {seed_text}")
    print("-" * 50)

    for temp in temperatures:
        print(f"Temperature: {temp}")
        generated = generate_text(model, seed_text, num_chars=200, temperature=temp)
        print(generated)
        print("-" * 50)
    print("\n")



In [ ]:
# Generate a longer text sample
long_sample = generate_text(
    model,
    "KING RICHARD III:\nNow is the winter of our discontent",
    num_chars=500,
    temperature=0.7
)
print("Long sample:")
print("-" * 50)
print(long_sample)

In [3]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.models import Sequential # Import Sequential from tensorflow.keras.models
from tensorflow.keras.layers import Dense

data, _ = tfds.load('tiny_shakespeare', split='train', with_info=True)
text = "".join([ex['text'].numpy().decode() for ex in data])

# Tokenize
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([text])
sequences = tokenizer.texts_to_sequences([text])[0]
char_index = tokenizer.word_index
index_char = {v: k for k, v in char_index.items()}

# Create input-output pairs
seq_length = 40
X, y = [], []
for i in range(len(sequences) - seq_length):
    X.append(sequences[i:i+seq_length])
    y.append(sequences[i+seq_length])
X = np.array(X)
y = tf.keras.utils.to_categorical(y, num_classes=len(char_index)+1)

# Model
model = Sequential([ # Now Sequential is defined and can be used
    tf.keras.layers.Embedding(input_dim=len(char_index)+1, output_dim=50, input_length=seq_length),
    tf.keras.layers.LSTM(128),
    Dense(len(char_index)+1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, batch_size=128, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 48s 6ms/step - loss: 2.2082
Epoch 2/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 79s 6ms/step - loss: 1.6559
Epoch 3/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - loss: 1.5409
Epoch 4/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 82s 5ms/step - loss: 1.4856
Epoch 5/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 43s 6ms/step - loss: 1.4514
Epoch 6/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - loss: 1.4292
Epoch 7/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 83s 6ms/step - loss: 1.4102
Epoch 8/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - loss: 1.3998
Epoch 9/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 81s 6ms/step - loss: 1.3861
Epoch 10/10
7843/7843 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - loss: 1.3764


In [ ]:
import matplotlib.pyplot as plt

# Compile with accuracy metric
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model and store training history
history = model.fit(X, y, batch_size=128, epochs=10)

# Plotting Loss and Accuracy
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss', color='red')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy', color='green')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
